In [ ]:
!pip install -U bitsandbytes
!pip install trl
!pip install datasets==3.6.0
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-rw7_3sii/unsloth_fc4b9047a7f54da0a7efc4ebba34972a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-rw7_3sii/unsloth_fc4b9047a7f54da0a7efc4ebba34972a
  Resolved https://github.com/unslothai/unsloth.git to commit ded942c765abf22fc8e3b0a67f847b83a6b2ad53
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 17.2 MB/s eta 0:00

In [ ]:
# Import Unsloth first for optimal performance
from unsloth import FastLanguageModel

import torch
import torch.nn as nn
from typing import Any
import pandas as pd
import tokenizers
from torch.utils.data import DataLoader, Dataset
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from trl import SFTTrainer
from datasets import load_dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# HYPERPARAMS
SEED = 42
LORA_RANK=32
BATCH_SIZE = 2
GRADIENT_ACCUMULATION=16
DATA_MAX_LEN = 1024
EPOCHS = 3
DROPOUT = 0.1
WEIGHT_DECAY = 0.01
TEMPERATURE = 0.8
GRAD_CLIP = 1 #0.3
WARMUP_RATIO = 0.01
LEARNING_RATE = 0.0002

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Use Unsloth's FastLanguageModel for optimized loading
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=DATA_MAX_LEN,
    dtype=None,  # Auto-detect dtype (bfloat16 or float16 based on GPU)
    load_in_4bit=True,  # Enable 4-bit quantization
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# Use Unsloth's FastLanguageModel.get_peft_model for optimized PEFT setup
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    lora_alpha=LORA_RANK * 2,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized gradient checkpointing
    random_state=SEED,
)

model.print_trainable_parameters()

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.3 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 25,231,360 || all params: 1,125,279,744 || trainable%: 2.2422


In [ ]:
def retrieve_data(dataset_name, tokenizer):
    dataset = load_dataset(dataset_name)

    def format_casehold(example):
        holdings = [
            example['holding_0'],
            example['holding_1'],
            example['holding_2'],
            example['holding_3'],
            example['holding_4']
        ]

        prompt = example['citing_prompt'] + "\n\nAnswer choices:\n"
        for i, holding in enumerate(holdings):
            prompt += f"{i}. {holding}\n"

        correct_idx = int(example['label'])
        example['question'] = prompt
        example['answer'] = holdings[correct_idx]

        return example

    dataset = dataset.map(format_casehold)

    train_dataset = dataset['train']
    test_dataset = dataset['test']

    return train_dataset, test_dataset

In [ ]:
train_dataset, test_dataset = retrieve_data('casehold/casehold', tokenizer)

# Use a subset of the dataset to speed up finetuning
train_dataset = train_dataset.select(range(1000))
test_dataset = test_dataset.select(range(1000))

training_args = TrainingArguments(
    output_dir="finetuned_model",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    max_grad_norm=GRAD_CLIP,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    optim="paged_adamw_8bit",
    warmup_ratio=WARMUP_RATIO,
    report_to="none",
    bf16=True, # Enable bfloat16 training
    load_best_model_at_end=True,
    gradient_checkpointing=True
)

def formatting_func(examples):
    questions = examples['question']
    answers = examples['answer']
    texts = []
    for question, answer in zip(questions, answers):
        text = f"{question}\n\nAnswer: {answer}"
        texts.append(text)
    return texts

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func,
    max_seq_length=DATA_MAX_LEN,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 96
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 25,231,360 of 1,125,279,744 (2.24% trained)


Epoch,Training Loss,Validation Loss
1,1.713400,1.707978
2,1.577200,1.705402


Epoch,Training Loss,Validation Loss
1,1.713400,1.707978
2,1.577200,1.705402
3,1.516200,1.713932


TrainOutput(global_step=96, training_loss=1.6167072455088298, metrics={'train_runtime': 803.9032, 'train_samples_per_second': 3.732, 'train_steps_per_second': 0.119, 'total_flos': 1.026338501701632e+16, 'train_loss': 1.6167072455088298, 'epoch': 3.0})

In [ ]:
import os
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

def save_as_gguf(model_dir, output_base_dir, quantize=True):
    """
    Merge LoRA weights and convert to GGUF format using Unsloth.

    Args:
        model_dir: Path to checkpoint with LoRA adapters
        output_base_dir: Base output directory (e.g., training_args.output_dir)
        quantize: Whether to create Q4 quantized version (recommended)
    """
    # Load the model directly using Unsloth's FastLanguageModel for saving
    # The model variable should still be accessible from previous cells
    global model
    global tokenizer

    gguf_path = os.path.join(output_base_dir, 'gguf')
    os.makedirs(gguf_path, exist_ok=True)

    # Unsloth's save_pretrained_gguf handles merging and conversion directly
    if quantize:
        q_type = "q4_k_m"
        output_file = os.path.join(gguf_path, 'model-q4.gguf')
        print("\n" + "="*60)
        print(f"SAVING AND QUANTIZING TO {q_type.upper()}")
        print("="*60)
    else:
        q_type = "f16"
        output_file = os.path.join(gguf_path, 'model-fp16.gguf')
        print("\n" + "="*60)
        print("SAVING AS FP16 GGUF")
        print("="*60)

    # Ensure model is on the correct device for saving, if needed
    # model.to("cpu") # Unsloth handles this internally usually

    # Use Unsloth's optimized function to save the model in GGUF format
    model.save_pretrained_gguf(
        output_file,
        tokenizer=tokenizer,
        quantization_method=q_type,
        # This ensures the adapter weights are merged before conversion
        # Unsloth's save_pretrained_gguf handles merging internally
    )

    if os.path.exists(output_file):
        file_size = os.path.getsize(output_file) / (1024**3)
        print(f"\n✓ GGUF saved to: {output_file} ({file_size:.2f} GB)")
        return output_file
    else:
        print("Error: GGUF file not found after conversion.")
        return None

model_dir = "finetuned_model/checkpoint-64"
# Use it:
gguf_model = save_as_gguf(
    model_dir=model_dir,
    output_base_dir=training_args.output_dir,
    quantize=True
)

print(f"\n✓ Model saved as GGUF: {gguf_model}")


SAVING AND QUANTIZING TO Q4_K_M
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:08<00:00,  8.41s/it]


Unsloth: Merge process complete. Saved to `/content/finetuned_model/gguf/model-q4.gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['tinyllama-cha

In [ ]:
# !zip -r /content/finetuned.zip /content/finetuned_model/gguf/
from google.colab import files
files.download('/content/finetuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>